In [1]:
import os
import sys
import torch
import radiomics
import torchio as tio
import SimpleITK as sitk

from glob import glob
from tqdm import tqdm
from radiomics.featureextractor import RadiomicsFeatureExtractor
from radiomics.imageoperations import checkMask

import logging
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

In [2]:
images = sorted([file for file in glob("../../data/RADCURE/volume+seg/*image*")])
masks = sorted([file for file in glob("../../data/RADCURE/volume+seg/*segmentation*")])

print(len(images), len(masks))

assert len(images) == len(masks)

subjects = [filename.split("/")[-1].split("_")[0] for filename in images]

all_masks_exist = True
for file in masks:
    if not os.path.exists(file):
        print(f"Mask does not exsist: {file}")
        all_masks_exist = False

if all_masks_exist:
    print("All masks exist")

all_imgs_exist = True
for file in images:
    if not os.path.exists(file):
        print(f"Image does not exsist: {file}")
        all_imgs_exist = False

if all_imgs_exist:
    print("All imgs exist")

1204 1204
All masks exist
All imgs exist


In [3]:
feature_list = []
subjects_list = []

paramFile = os.path.abspath("./Params-RADCURE.yaml")

feature_extractor = RadiomicsFeatureExtractor(paramFile)
# resample = tio.transforms.Resample(target=1.0)

for idx in tqdm(range(len(images))):       
    
    try:
        img = tio.ScalarImage(images[idx])
        seg = tio.transforms.Resample(target=img)(tio.LabelMap(masks[idx]))
        
        subject = tio.Subject(volume=img, segmentation=seg)

        # subject = resample(subject)
        features = feature_extractor.execute(subject.volume.as_sitk(), subject.segmentation.as_sitk(), label=255)
        feature_list.append(features)
        subjects_list.append(subjects[idx])

    except Exception as e:
        print(e)        
        print(images[idx])
        continue

torch.save(feature_list, f"./radiomics_features_RADCURE_new2.pt")
torch.save(subjects_list, f"./subjects_RADCURE_new2.pt")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1204/1204 [3:15:45<00:00,  9.76s/it]


In [4]:
len(features.keys())

1283

In [5]:
original_features = []
wavelet_features = []
log_features = []
others = []

for feature in features.keys():
    if feature.startswith("original"):
        original_features.append(feature)

    elif feature.startswith("wavelet"):
        wavelet_features.append(feature)

    elif feature.startswith("log"):
        log_features.append(feature)

    else:
        others.append(feature)

In [6]:
len(original_features)

102

In [7]:
len(wavelet_features)

704

In [8]:
len(log_features)

440

In [9]:
len(original_features) + len(wavelet_features) + len(log_features)

1246